In [1]:
# Download and load the Titanic dataset for EDA demonstration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Download the Titanic dataset from a reliable source
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nDataset Info:")
print(df.info())
print("\nFirst 5 rows:")
print(df.head())

Dataset Shape: (891, 12)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None

First 5 rows:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4   

In [2]:
# Comprehensive Exploratory Data Analysis - Missing Values Analysis
print("=" * 60)
print("MISSING VALUES ANALYSIS")
print("=" * 60)

# Check for missing values
missing_values = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percentage': missing_percentage
})

print(missing_df[missing_df['Missing Count'] > 0])

print("\n" + "=" * 60)
print("DESCRIPTIVE STATISTICS")
print("=" * 60)

# Descriptive statistics for numerical columns
print("\nNumerical Features:")
print(df.describe())

print("\nCategorical Features:")
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f"\n{col}:")
    print(df[col].value_counts())

print("\n" + "=" * 60)
print("TARGET VARIABLE DISTRIBUTION")
print("=" * 60)

survival_counts = df['Survived'].value_counts()
survival_percentage = df['Survived'].value_counts(normalize=True) * 100

print("Survival Distribution:")
print(f"Did not survive (0): {survival_counts[0]} ({survival_percentage[0]:.1f}%)")
print(f"Survived (1): {survival_counts[1]} ({survival_percentage[1]:.1f}%)")

# Data types and unique values
print("\n" + "=" * 60)
print("DATA TYPES AND UNIQUE VALUES")
print("=" * 60)

for col in df.columns:
    print(f"{col}: {df[col].dtype}, Unique values: {df[col].nunique()}")

MISSING VALUES ANALYSIS
          Missing Count  Missing Percentage
Age                 177           19.865320
Cabin               687           77.104377
Embarked              2            0.224467

DESCRIPTIVE STATISTICS

Numerical Features:
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.69342

In [3]:
# Data cleaning and feature engineering for better analysis
print("=" * 60)
print("DATA CLEANING AND FEATURE ENGINEERING")
print("=" * 60)

# Create a copy for analysis
df_clean = df.copy()

# 1. Handle missing Age values - fill with median grouped by Pclass and Sex
df_clean['Age'] = df_clean.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))

# 2. Handle missing Embarked values - fill with mode
df_clean['Embarked'] = df_clean['Embarked'].fillna(df_clean['Embarked'].mode()[0])

# 3. Create age groups for better analysis
df_clean['AgeGroup'] = pd.cut(df_clean['Age'], bins=[0, 12, 18, 35, 60, 100],
                              labels=['Child', 'Teen', 'Adult', 'Middle-aged', 'Senior'])

# 4. Create family size feature
df_clean['FamilySize'] = df_clean['SibSp'] + df_clean['Parch'] + 1

# 5. Create fare bins
df_clean['FareBin'] = pd.cut(df_clean['Fare'], bins=4, labels=['Low', 'Medium', 'High', 'Very High'])

# 6. Extract title from name
df_clean['Title'] = df_clean['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# Group less common titles
title_mapping = {
    'Mr': 'Mr', 'Miss': 'Miss', 'Mrs': 'Mrs', 'Master': 'Master',
    'Dr': 'Other', 'Rev': 'Other', 'Mlle': 'Miss', 'Major': 'Other',
    'Col': 'Other', 'Sir': 'Other', 'Don': 'Other', 'Mme': 'Mrs',
    'Jonkheer': 'Other', 'Lady': 'Other', 'Capt': 'Other', 'Countess': 'Other',
    'Ms': 'Miss', 'Dona': 'Other'
}
df_clean['Title'] = df_clean['Title'].map(title_mapping)

print("Missing values after cleaning:")
print(df_clean.isnull().sum())

print("\nNew features created:")
print(f"AgeGroup: {df_clean['AgeGroup'].value_counts()}")
print(f"\nFamilySize statistics: {df_clean['FamilySize'].describe()}")
print(f"\nTitle distribution: {df_clean['Title'].value_counts()}")

# Save cleaned data for visualization
df_clean.to_csv('titanic_cleaned.csv', index=False)
print("\nCleaned dataset saved as 'titanic_cleaned.csv'")

DATA CLEANING AND FEATURE ENGINEERING
Missing values after cleaning:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
AgeGroup         0
FamilySize       0
FareBin          0
Title            0
dtype: int64

New features created:
AgeGroup: AgeGroup
Adult          514
Middle-aged    216
Teen            70
Child           69
Senior          22
Name: count, dtype: int64

FamilySize statistics: count    891.000000
mean       1.904602
std        1.613459
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max       11.000000
Name: FamilySize, dtype: float64

Title distribution: Title
Mr        517
Miss      185
Mrs       126
Master     40
Other      23
Name: count, dtype: int64

Cleaned dataset saved as 'titanic_cleaned.csv'


In [15]:
# Statistical analysis and correlation analysis
print("=" * 60)
print("CORRELATION ANALYSIS")
print("=" * 60)

# Feature Engineering: Create FamilySize, AgeGroup, and Title columns
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80]
labels = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80']
df['AgeGroup'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df['Title'] = df['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df['Title'] = df['Title'].replace('Mlle', 'Miss')
df['Title'] = df['Title'].replace('Ms', 'Miss')
df['Title'] = df['Title'].replace('Mme', 'Mrs')


# Select numerical columns for correlation analysis
numerical_cols = ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize']
correlation_matrix = df[numerical_cols].corr()

print("Correlation Matrix:")
print(correlation_matrix.round(3))

print("\nCorrelations with Survival (sorted by absolute value):")
survival_correlations = correlation_matrix['Survived'].drop('Survived').abs().sort_values(ascending=False)
for feature, corr in survival_correlations.items():
    direction = "positive" if correlation_matrix['Survived'][feature] > 0 else "negative"
    print(f"{feature}: {correlation_matrix['Survived'][feature]:.3f} ({direction})")

# Survival rates by different categories
print("\n" + "=" * 60)
print("SURVIVAL RATES BY CATEGORIES")
print("=" * 60)

# Survival by Sex
print("Survival by Sex:")
sex_survival = df.groupby('Sex')['Survived'].agg(['count', 'sum', 'mean'])
sex_survival.columns = ['Total', 'Survived', 'Survival_Rate']
print(sex_survival)

print("\nSurvival by Passenger Class:")
pclass_survival = df.groupby('Pclass')['Survived'].agg(['count', 'sum', 'mean'])
pclass_survival.columns = ['Total', 'Survived', 'Survival_Rate']
print(pclass_survival)

print("\nSurvival by Embarked Port:")
embarked_survival = df.groupby('Embarked')['Survived'].agg(['count', 'sum', 'mean'])
embarked_survival.columns = ['Total', 'Survived', 'Survival_Rate']
print(embarked_survival)

print("\nSurvival by Age Group:")
age_survival = df.groupby('AgeGroup')['Survived'].agg(['count', 'sum', 'mean'])
age_survival.columns = ['Total', 'Survived', 'Survival_Rate']
print(age_survival)

print("\nSurvival by Title:")
title_survival = df.groupby('Title')['Survived'].agg(['count', 'sum', 'mean'])
title_survival.columns = ['Total', 'Survived', 'Survival_Rate']
print(title_survival)

CORRELATION ANALYSIS
Correlation Matrix:
            Survived  Pclass    Age  SibSp  Parch   Fare  FamilySize
Survived       1.000  -0.338 -0.060 -0.035  0.082  0.257       0.017
Pclass        -0.338   1.000 -0.414  0.083  0.018 -0.549       0.066
Age           -0.060  -0.414  1.000 -0.250 -0.176  0.123      -0.258
SibSp         -0.035   0.083 -0.250  1.000  0.415  0.160       0.891
Parch          0.082   0.018 -0.176  0.415  1.000  0.216       0.783
Fare           0.257  -0.549  0.123  0.160  0.216  1.000       0.217
FamilySize     0.017   0.066 -0.258  0.891  0.783  0.217       1.000

Correlations with Survival (sorted by absolute value):
Pclass: -0.338 (negative)
Fare: 0.257 (positive)
Parch: 0.082 (positive)
Age: -0.060 (negative)
SibSp: -0.035 (negative)
FamilySize: 0.017 (positive)

SURVIVAL RATES BY CATEGORIES
Survival by Sex:
        Total  Survived  Survival_Rate
Sex                                   
female    314       233       0.742038
male      577       109       0.18890

/tmp/ipython-input-2714376520.py:53: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [5]:
# Let's create the data in the right format for the visualization tool
import json

# Create histogram data for age distribution
age_data = df_clean['Age'].dropna()
age_hist, age_bins = np.histogram(age_data, bins=25)

# Create data structure for histogram
hist_data = []
for i in range(len(age_hist)):
    hist_data.extend([age_bins[i]] * age_hist[i])

print("Age distribution data prepared")
print(f"Mean age: {age_data.mean():.2f}")
print(f"Total records: {len(hist_data)}")
print(f"Age range: {age_data.min():.2f} - {age_data.max():.2f}")

# Save the data
age_distribution_data = {
    "age": hist_data,
    "mean_age": float(age_data.mean())
}

with open('age_distribution_data.json', 'w') as f:
    json.dump(age_distribution_data, f)

print("Age distribution data saved to age_distribution_data.json")

Age distribution data prepared
Mean age: 29.11
Total records: 891
Age range: 0.42 - 80.00
Age distribution data saved to age_distribution_data.json


In [2]:
import pandas as pd
import plotly.graph_objs as go

# Load data
df = pd.read_csv('titanic_cleaned.csv')

# Color settings
bar_color = 'rgba(31,184,205,0.7)'  # strong cyan with alpha 0.7
mean_line_color = '#DB4545'  # bright red, brand color 2

# Calculate mean age
mean_age = df['Age'].mean()

fig = go.Figure()

# Histogram with 25 bins, semi-transparent
fig.add_trace(go.Histogram(
    x=df['Age'].dropna(),
    nbinsx=25,
    marker_color=bar_color,
    hovertemplate='Age: %{x}<br>Freq: %{y}<extra></extra>'
))

# Mean age vertical line and label (using shape and annotation)
fig.add_vline(
    x=mean_age,
    line_dash="dash",
    line_color=mean_line_color,
    line_width=2
)
fig.add_trace(go.Scatter(
    x=[mean_age+1],
    y=[0.95*df['Age'].notnull().sum()/25],
    mode='text',
    text=["Mean Age"],
    textposition='middle left',
    textfont=dict(color=mean_line_color, size=14),
    showlegend=False,
    hoverinfo='skip'
))

fig.update_layout(
    title_text='AgeDistrib TitanicPsg',
    xaxis_title='Age (years)',
    yaxis_title='Frequency',
    showlegend=False,
)

fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True)




In [4]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

data = pd.read_csv('titanic_cleaned.csv')
cols = ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize']
corr_matrix = data[cols].corr().round(2)

z = corr_matrix.values
x = corr_matrix.columns.tolist()
y = corr_matrix.index.tolist()

# Create custom color scale centered at 0 with diverging RdBu colors
color_scale = [
    [0.0, '#DB4545'],   # bright red
    [0.5, '#D2BA4C'],  # moderate yellow (for midpoint)
    [1.0, '#1FB8CD']   # strong cyan
]

# Custom text annotation for the cells, max 4 chars incl. decimal
text = np.vectorize(lambda v: f"{v:.2f}")(z)

fig = go.Figure(data=go.Heatmap(
    z=z,
    x=x,
    y=y,
    text=text,
    texttemplate="%{text}",
    colorscale='RdBu',
    zmid=0,
    colorbar=dict(title='Corr.'),
    showscale=True,
    hovertemplate="%{x}<br>%{y}: %{z:.2f}<extra></extra>"
))

fig.update_layout(
    title='Correlation Matrix - Titanic Dataset Features',
)
fig.update_xaxes(title='Feature', tickangle=0)
fig.update_yaxes(title='Feature', tickangle=0)


In [6]:
import pandas as pd
import plotly.graph_objects as go

# Load the data
f = 'titanic_cleaned.csv'
df = pd.read_csv(f)

# Count of survived by sex
count_df = df.groupby(['Sex', 'Survived']).size().unstack(fill_value=0)

# Set up data for plotting
x = count_df.index.tolist()
not_survived = count_df[0].tolist()
survived = count_df[1].tolist()

# Use the first two brand colors for not survived (red) and survived (cyan)
colors = ['#DB4545', '#1FB8CD']

fig = go.Figure()
fig.add_bar(x=x, y=not_survived, name='Not Surv', marker_color=colors[0], cliponaxis=False)
fig.add_bar(x=x, y=survived, name='Survived', marker_color=colors[1], cliponaxis=False)

fig.update_layout(
    barmode='group',
    title_text='Survival Count by Gender',
    xaxis_title='Sex',
    yaxis_title='Count',
    legend=dict(orientation='h', yanchor='bottom', y=1.05, xanchor='center', x=0.5)
)
fig.update_xaxes(title_text='Sex')
fig.update_yaxes(title_text='Count')



In [9]:
import pandas as pd
import plotly.graph_objs as go

df = pd.read_csv("titanic_cleaned.csv")
class_map = {1: '1st Class', 2: '2nd Class', 3: '3rd Class'}
df['Pclass_label'] = df['Pclass'].map(class_map)
counts = df.groupby(['Pclass_label', 'Survived']).size().reset_index(name='Count')
counts['Survived'] = counts['Survived'].map({0: 'Not survived', 1: 'Survived'})

colors = ['#DB4545', '#2E8B57']

fig = go.Figure()
for i, survived_status in enumerate(['Not survived', 'Survived']):
    data = counts[counts['Survived'] == survived_status]
    fig.add_bar(
        x=data['Pclass_label'],
        y=data['Count'],
        name=survived_status,
        marker_color=colors[i],
        cliponaxis=False
    )

fig.update_layout(
    title='Survival Count by Passenger Class',
    xaxis_title='Passenger Class',
    yaxis_title='Count',
    legend=dict(orientation='h', yanchor='bottom', y=1.05, xanchor='center', x=0.5)
)




In [11]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('titanic_cleaned.csv')
surv_map = {0: 'Did not survive', 1: 'Survived'}
df['Survival Group'] = df['Survived'].map(surv_map)

fig = px.box(
    df,
    x='Survival Group',
    y='Fare',
    color='Survival Group',
    color_discrete_sequence=['#1FB8CD', '#DB4545']
)
fig.update_xaxes(title_text='Survival Status', tickvals=['Did not survive','Survived'])
fig.update_yaxes(title_text='Fare (£)', range=[0, 200])
fig.update_layout(title_text='Fare Distribution by Survival Status')
# Legend center since only two items
fig.update_layout(legend=dict(orientation='h', yanchor='bottom', y=1.05, xanchor='center', x=0.5))



In [8]:
!pip install -U kaleido